# Tensorflow 2 特征工程结构化数据文档

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## read the data
> print some samples

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


# read the data
> 分割数据集包括训练集、验证集和测试集

In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train_examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train_examples
49 validation examples
61 test examples


## 使用tf.data 创建input pipeline

In [4]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    """
    帮助函数 返回tf.data的数据集
    """
    df = df.copy()
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

### 超参数设置 并打印部分tf.data的数据

In [5]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, batch_size=batch_size)
test_ds = df_to_dataset(test, batch_size=batch_size)

In [11]:
for feature_map, label_batch in train_ds.take(1):
    print("Every Feature: ", list(feature_map.keys()))
    print("A batch of ages: ", feature_map['age'])
    print("A batch of thal", feature_map['thal'])
    print("A batch of target: ", label_batch)

Every Feature:  ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages:  tf.Tensor([58 63 43 38 50], shape=(5,), dtype=int32)
A batch of thal tf.Tensor([b'reversible' b'normal' b'reversible' b'reversible' b'normal'], shape=(5,), dtype=string)
A batch of target:  tf.Tensor([1 0 1 1 0], shape=(5,), dtype=int32)


## 特征列示例

### 1) 数值型属性处理 Numeric column

In [7]:
example_batch = next(iter(train_ds))[0]
# 帮助函数 创建一个特征实例 并转换
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())
age = feature_column.numeric_column('age')
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[52.]
 [59.]
 [56.]
 [52.]
 [62.]]


### 2) 连续型属性处理 离散化处理 比如年龄 按照区间进行划分 Bucketized columns

In [8]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


### 3）种类型属性处理  Categorical columns

In [9]:
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


### 4) 如果某列含有上千种类别 使用独热编码不合适 可以使用embedding column 压缩维度 Embedding columns

In [12]:
# embedding columns 的输入是categorical column
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[-0.6725974   0.13393505 -0.43921667 -0.5595472  -0.02276379  0.06387953
   0.20338741 -0.22088109]
 [-0.6725974   0.13393505 -0.43921667 -0.5595472  -0.02276379  0.06387953
   0.20338741 -0.22088109]
 [ 0.4448062  -0.12826684 -0.16934529 -0.33862525 -0.4983676   0.55637455
   0.32413128 -0.27672908]
 [-0.6725974   0.13393505 -0.43921667 -0.5595472  -0.02276379  0.06387953
   0.20338741 -0.22088109]
 [-0.6725974   0.13393505 -0.43921667 -0.5595472  -0.02276379  0.06387953
   0.20338741 -0.22088109]]


### 5）Hashed feature columns

In [16]:
thal_hashed = feature_column.categorical_column_with_hash_bucket('thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 6）Crossed feature columns  将几个特征组合成一个特征、模型可以对每一个特征组合学习独立的权重

In [18]:
crossed_features = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_features))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 选出合适的列训练模型

In [19]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))


# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

## 创建特征层

In [32]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
    
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)
model.fit(train_ds, validation_data=val_ds, epochs=5)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/5
39/39 [==============================] - 1s 28ms/step - loss: 1.6773 - accuracy: 0.5544 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
39/39 [==============================] - 1s 25ms/step - loss: 0.9423 - accuracy: 0.6788 - val_loss: 0.3304 - val_accuracy: 0.8980loss: 0.9814 - accuracy - ETA: 0s - loss: 0.9394 - accuracy: 0.68
Epoch 3/5
39/39 [==============================] - 1s 26ms/step - loss: 0.6669 - accuracy: 0.6995 - val_loss: 0.3408 - val_accuracy: 0.8980
Epoch 4/5
39/39

## evaluate the model

In [33]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

13/13 [==============================] - 0s 16ms/step - loss: 0.5519 - accuracy: 0.7213
Accuracy 0.72131145
